<a href="https://colab.research.google.com/github/Sanskriti0204/PracticeCodes/blob/main/IMDB%20rating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data_yelp=pd.read_csv('yelp_labelled.txt',sep='\t',header=None)
data_yelp.head()

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [4]:
column_name=['Review','Sentiment']
data_yelp.columns=column_name
data_yelp.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [5]:
data_amazon=pd.read_csv('amazon_cells_labelled.txt',sep='\t',header=None)
data_amazon.head()

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [6]:
data_amazon.columns=column_name
data_amazon.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [7]:
data_amazon.shape

(1000, 2)

In [8]:
data_imdb=pd.read_csv('imdb_labelled.txt',sep='\t',header=None)
data_imdb.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [9]:
data_imdb.columns=column_name
data_imdb.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [13]:
# Instead of using append, use pd.concat to combine the DataFrames:
data = pd.concat([data_yelp, data_amazon, data_imdb], ignore_index=True)
data.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [14]:
data['Sentiment'].value_counts()

,count
Sentiment,
1,1386
0,1362


In [16]:
data.isnull().sum()

,0
Review,0
Sentiment,0


In [17]:
x=data['Review']
y=data['Sentiment']

Cleaning of Data

In [21]:
import string
punct=string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [22]:
from spacy.lang.en.stop_words import STOP_WORDS
stopwords=list(STOP_WORDS)

In [25]:
import spacy
nlp=spacy.load('en_core_web_sm')
def text_data_cleaning(sentence):
  doc=nlp(sentence)

  tokens=[]
  for token in doc:
    if token.lemma_!='-PRON-':
      temp=token.lemma_.lower().strip()
    else:
      temp=token.lower_
    tokens.append(temp)
  cleaned_tokens=[]
  for token in tokens:
    if token not in stopwords and token not in punct:
      cleaned_tokens.append(token)
  return cleaned_tokens

In [26]:
text_data_cleaning("Hello all, It's a beautiful day outside there!")

['hello', 'beautiful', 'day', 'outside']

In [28]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [29]:
tfidf=TfidfVectorizer(tokenizer=text_data_cleaning)
classifier=LinearSVC()

In [30]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)


In [31]:
x_train.shape,x_test.shape

((2198,), (550,))

In [32]:
clf=Pipeline([('tfidf',tfidf),('clf',classifier)])
clf.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x78a389184f70>)),
                ('clf', LinearSVC())])

Prediction

In [36]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [38]:
y_pred=clf.predict(x_test)

In [39]:
confusion_matrix(y_test,y_pred)

array([[201,  78],
       [ 50, 221]])

In [40]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.80      0.72      0.76       279
           1       0.74      0.82      0.78       271

    accuracy                           0.77       550
   macro avg       0.77      0.77      0.77       550
weighted avg       0.77      0.77      0.77       550



In [41]:
clf.predict(['Wow, i am learning Natural Language Processing in fun fashion!'])

array([1])

In [42]:
clf.predict(['Its hard to learn new things'])

array([0])

In [ ]:
|